In [1]:
using PandaRobot
using GuSTO

INFO: Loading HttpServer methods...


HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/e38c1e09106b4d8eef7b28477203db186cf7ad2e-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/e38c1e09106b4d8eef7b28477203db186cf7ad2e-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

In [2]:
robot = PandaBot()
env = Factory()

th_init = [0;0;0; -0.1; 0; 0.5; 0]
th_goal = [0.5;0.5;0.5; -1.; 0; 0.5; 0]

th_init = zeros(robot.num_joints)
th_goal = [pi/4; 0; pi/6; 0; pi/4; 0; pi/7]

RigidBodyDynamics.set_configuration!(robot.state, th_goal)
point_inWorldFrame = get_EE_position(robot);

In [3]:
env = Factory()

x_init, x_goal = zeros(2*robot.num_joints), zeros(2*robot.num_joints)
for i in 1:robot.num_joints
    x_init[2*i-1:2*i] = [cos(th_init[i]); sin(th_init[i])]
    x_goal[2*i-1:2*i] = [cos(th_goal[i]); sin(th_goal[i])]
end

model = PandaKin([i for i in point_inWorldFrame])
N = 80
tf_guess = 50.

50.0

In [ ]:
PD = ProblemDefinition(robot, model, env, x_init, x_goal)
TOPgusto = TrajectoryOptimizationProblem(PD, N, tf_guess, fixed_final_time=true)
TOSgusto = TrajectoryOptimizationSolution(TOPgusto)
solve_SCP!(TOSgusto, TOPgusto, solve_gusto_cvx!, init_traj_so1, "Gurobi", OutputFlag=0)

In [ ]:
TOSgusto.SCPS.successful

In [ ]:
θf = get_configuration(TOSgusto.traj.X[:,end], model)
RigidBodyDynamics.set_configuration!(robot.state, θf)
r_EE_f = get_EE_position(robot)
println("Distance between r_EE_goal and solution is ", norm(r_EE_f-point_inWorldFrame))
abs.(r_EE_f-point_inWorldFrame)
[r_EE_f point_inWorldFrame]

In [ ]:
fieldnames(TOSgusto.SCPS.param.alg)
TOSgusto.SCPS.param.alg.Delta_vec
TOSgusto.SCPS.param.convergence_threshold
TOSgusto.SCPS.convergence_measure
TOSgusto.SCPS.param.alg.trust_region_satisfied_vec
TOSgusto.SCPS.param.alg.convex_ineq_satisfied_vec
TOSgusto.SCPS.successful
fieldnames(TOSgusto.SCPS)
TOSgusto.SCPS.total_time

In [ ]:
norms = zeros(model.num_joints,N)
for k in 1:N
   for idx in 1:model.num_joints
        norms[idx,k] = sqrt(TOSgusto.traj.X[2*idx-1,k]^2 + TOSgusto.traj.X[2*idx,k]^2)
    end
end

for idx in 1:model.num_joints
    PyPlot.plot(norms[idx,:])
end

In [ ]:
joint_traj = zeros(robot.num_joints,N)
for k in 1:N
    joint_traj[:,k] = get_configuration(TOSgusto.traj.X[:,k],model)
end

In [ ]:
using MeshCat
using MeshCatMechanisms

using RigidBodySim
using RigidBodyDynamics

vis = Visualizer()

pd = robot.pan
mvis = MechanismVisualizer(
    pd.mechanism,
    URDFVisuals(PandaRobot.urdfpath(), package_path=[dirname(dirname(PandaRobot.urdfpath()))]),
    vis);

vis[:obstacles]
for (idx,obs) in enumerate(env.obstacle_set)
    setobject!(vis[:obstacles][Symbol(string("obs",idx))], 
        Object(obs,MeshBasicMaterial(color=RGBA(1.0,0.,0.,0.3))))
end

q = Vector{Array{Float64,1}}(0)
for k in 1:N
    push!(q, joint_traj[:,k]) 
end

setanimation!(mvis, 1:length(q), q)

plot_in_cell = true
plot_in_cell ? IJuliaCell(vis) : open(vis)
#sleep(1)

In [ ]:
for k in 1:model.num_joints
    PyPlot.plot(joint_traj[k,:])
end